# Week 3 Assignment Solutions

### Question 1: Obtain the data in the table of postal codes and to transform the data into a pandas dataframe 

In [1]:
# import the library we use to open URLS & specify which URL/web page we are going to be scraping
#from urllib.request import urlopen
import urllib.request
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [2]:
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)

In [3]:
# import the BeautifulSoup library so we can parse the HTML from our URL into the BeautifulSoup parse tree format
from bs4 import BeautifulSoup 
soup = BeautifulSoup(page, "lxml")

In [4]:
# use the 'find_all' function to bring back all instances of the 'table' tag in
all_tables=soup.find_all("table")

In [5]:
right_table=soup.find('table', class_='wikitable sortable')

In [6]:
P=[]
B=[]
N=[]

for row in right_table.find_all('tr'):
   # cells = row.replace('\n', '')
    cells=row.find_all('td')
    
    if len(cells)==3 and cells[1].find(text = True) != 'Not assigned':
        P.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        N.append(cells[2].find(text=True))

In [7]:
import pandas as pd
df =pd.DataFrame(P,columns=['Postal Code'])
df['Borough']= B
df['Neighbourhood'] = N

#### Clean the data

In [13]:
# Remove the Not Assigned attributes in Borough
index=df[df['Borough']== 'Not assigned'].index
df.drop(index, inplace=True)
df.reset_index(drop = True, inplace = True)
df = df.replace('\n','', regex=True)

Borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [14]:
df['Neighbourhood']=df['Neighbourhood'].replace('Not assigned', df['Borough'])

### Question 2: Used the .shape method to print the number of rows of your dataframe

In [15]:
print(df.shape)
df.head(12)

(103, 3)


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Use the Geocoder package or the csv file to create dataframe

In [16]:
#Install geopy
!pip install geocoder

In [17]:
import pandas as pd
geo_df = pd.read_csv (r'C:\Users\smwemeke\Desktop\Geospatial_Coordinates.csv', header=0)
print (geo_df)

    Postal Code   Latitude  Longitude
0           M1B  43.806686 -79.194353
1           M1C  43.784535 -79.160497
2           M1E  43.763573 -79.188711
3           M1G  43.770992 -79.216917
4           M1H  43.773136 -79.239476
..          ...        ...        ...
98          M9N  43.706876 -79.518188
99          M9P  43.696319 -79.532242
100         M9R  43.688905 -79.554724
101         M9V  43.739416 -79.588437
102         M9W  43.706748 -79.594054

[103 rows x 3 columns]


#### Merge the 2 dataframes

In [18]:
latlng = pd.merge(df,geo_df, on='Postal Code', how='outer')

latlng.head(12)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Question 3: Consider only Toronto Boroughs

In [19]:
df_toronto = latlng[latlng['Borough'].str.contains("Toronto")]
df_toronto.reset_index(drop=True, inplace=True)
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


#### Import necessary Libraries

In [20]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library


print('Libraries imported.')

Libraries imported.


### Map Toronto Neighborhoods

In [21]:
address = 'Toronto, Canada'

latitude = 43.7166
longitude = -79.3407
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

The geograpical coordinate of Toronto are 43.7166, -79.3407.


### Cluster Neighborhoods

In [22]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = df_toronto.loc[:,['Latitude','Longitude']] #consider only longitude and latitude
                                 
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 1, 0, 0, 3, 0, 2])

In [23]:
# add clustering labels to dataframe
df_toronto.insert(0, 'Cluster Labels', kmeans.labels_)
df_toronto.head()

,Cluster Labels,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,0,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,0,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,0,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,1,M4E,East Toronto,The Beaches,43.676357,-79.293031


### Generate clusterized map

In [24]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighbourhood'], df_toronto['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters